In [1]:
import pandas as pd
import numpy as np

from collections import Counter

In [2]:
recommend_problems = pd.read_pickle("C:/algorithm/new_data/recommend_problems.pkl")
algorithm = pd.read_pickle("C:/algorithm/new_data/algorithms.pkl")

rankers = pd.read_pickle("C:/algorithm/data/rankers.pkl")
ranker_problems = pd.read_pickle("C:/algorithm/data/ranker_problems.pkl")

In [3]:
recommend_problems.info()
recommend_problems.head()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 9283 entries, 0 to 18854
Columns: 179 entries, id to algo_cnt
dtypes: float64(1), int64(176), object(2)
memory usage: 12.7+ MB


,id,name,information,correct_user,submission_cnt,correct_rate,수학,다이나믹 프로그래밍,구현,그래프 이론,...,델로네 삼각분할,4차원 이상의 기하학,로프,춤추는 링크,크누스 X,접미사 트리,레드-블랙 트리,탑 트리,이산 k제곱근,algo_cnt
0,1000,A+B,"다국어,디버그,분류",118169,381174,43.753,1,0,1,0,...,0,0,0,0,0,0,0,0,0,3
1,1001,A-B,"디버그,분류",97109,159168,72.049,1,0,1,0,...,0,0,0,0,0,0,0,0,0,3
2,1002,터렛,분류,16141,105843,20.562,1,0,0,0,...,0,0,0,0,0,0,0,0,0,2
3,1003,피보나치 함수,분류,23293,113982,30.087,0,1,0,0,...,0,0,0,0,0,0,0,0,0,1
4,1004,어린 왕자,분류,6503,19933,41.555,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1


In [8]:
sum(recommend_problems != recommend_problems.unique())

0

In [19]:
ranker_problems.head()

,id,user_id,user,problem_id,TF
0,0,0,koosaga,1000,o
1,1,0,koosaga,1001,o
2,2,0,koosaga,1002,o
3,3,0,koosaga,1003,o
4,4,0,koosaga,1004,o


In [34]:
ranker_problems['score'] = ranker_problems['TF'].replace({'o' : 0, 'x': 2})

In [35]:
ranker_problems['problem_id'] = ranker_problems['problem_id'].astype('int64').astype(str)

In [36]:
ranker_problems.tail()

,id,user_id,user,problem_id,TF,score
6322241,6322241,34899,hsy0113,11722,o,0
6322242,6322242,34899,hsy0113,11726,o,0
6322243,6322243,34899,hsy0113,11727,o,0
6322244,6322244,34899,hsy0113,1330,x,2
6322245,6322245,34899,hsy0113,11055,x,2


In [37]:
problems = ranker_problems['problem_id'].unique()
len(problems)

15774

In [38]:
table = ranker_problems.pivot_table(index='user_id', columns='problem_id', values='score')

In [39]:
table

problem_id,1000,10000,10001,10003,10004,10005,10006,10007,10008,1001,...,9990,9991,9992,9993,9994,9995,9996,9997,9998,9999
user_id,,,,,,,,,,,,,,,,,,,,,
0,0.0,0.0,0.0,0.0,0.0,NaN,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,NaN,0.0,NaN,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,0.0,...,NaN,0.0,0.0,0.0,0.0,NaN,0.0,2.0,0.0,0.0
2,0.0,NaN,NaN,0.0,0.0,NaN,0.0,0.0,0.0,0.0,...,NaN,0.0,NaN,0.0,0.0,NaN,0.0,NaN,NaN,0.0
3,0.0,NaN,0.0,NaN,0.0,NaN,0.0,NaN,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,NaN,0.0
4,0.0,0.0,NaN,NaN,0.0,NaN,NaN,NaN,NaN,0.0,...,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
34895,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
34896,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
34897,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [40]:
matrix = table.reset_index()

In [41]:
matrix = matrix.fillna(1)

In [42]:
matrix

problem_id,user_id,1000,10000,10001,10003,10004,10005,10006,10007,10008,...,9990,9991,9992,9993,9994,9995,9996,9997,9998,9999
0,0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0
1,1,0.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,0.0,0.0,0.0,0.0,1.0,0.0,2.0,0.0,0.0
2,2,0.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,...,1.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0
3,3,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,...,1.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,1.0,0.0
4,4,0.0,0.0,1.0,1.0,0.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
34895,34895,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
34896,34896,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
34897,34897,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
34898,34898,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
